In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Read BlastX DIAMOND results

assembelies were downloaded from different sources then went through:
1) Prodigal
2) CD-HIT
3) DIAMOND (against the antiSMASH AD domains)

In [99]:
#Random sampling of ORFS to start with a similar number of ORFs:

In [400]:
nrps_soil=pd.read_table("C:/Users/barakd/Desktop/metagenomes/soil_AD_random.txt")
nrps_roots=pd.read_table("C:/Users/barakd/Desktop/metagenomes/roots_AD_random.txt")
pks_soil=pd.read_table("C:/Users/barakd/Desktop/metagenomes/soil_KS_random.txt")
pks_roots=pd.read_table("C:/Users/barakd/Desktop/metagenomes/roots_KS_random.txt")

In [401]:
columns=["query", "accession", "query_length","subject_length","alignment_length","query_start","query_end","identity", "eval", "bitscore","sample"]
pks_roots.columns=columns
pks_soil.columns=columns
nrps_soil.columns=columns
nrps_roots.columns=columns

In [402]:
def prepare_diamond(table):
    table["genes"]=table["accession"].str[40:]
    table["accession"]=table['accession'].map(lambda x: str(x)[:10])
    table_flted=table[table["alignment_length"]>60]
    table_flted=table_flted[table_flted["eval"]<0.0000000001]
    table_flted=table_flted[table_flted["query_length"]>200]
    return table_flted

In [403]:
soil_nrps_fltd=prepare_diamond(nrps_soil)
soil_pks_fltd=prepare_diamond(pks_soil)
roots_nrps_fltd=prepare_diamond(nrps_roots)
roots_pks_fltd=prepare_diamond(pks_roots)

In [550]:
megan_soil_nrps.head()

,query,taxonomy
0,scaffold_330881_c1_1,root;
1,scaffold_970497_c1_1,root;
2,Ga0075525_10243375_1,root;
3,Ga0075529_1092363_1,root;
4,Ga0075525_10739222_1,root;


In [335]:
#Read the MEGAN of ALL mined NRPS/PKS within soil/roots, and assigne annotations only for the randomly chosen ones:
megan_soil_pks=pd.read_table("C:/Users/barakd/Desktop/metagenomes/soil_pks_megan.txt")
megan_soil_nrps=pd.read_table("C:/Users/barakd/Desktop/metagenomes/soil_nrps_megan.txt")
megan_roots_pks=pd.read_table("C:/Users/barakd/Desktop/metagenomes/roots_pks_megan.txt")
megan_roots_nrps=pd.read_table("C:/Users/barakd/Desktop/metagenomes/roots_nrps_megan.txt")

In [2]:
def prepare_megan(table):
    table.columns=["query", "taxonomy"]
    table_filter=table["taxonomy"].str.split(pat=';', expand=True)
    table_filter["query"]=table["query"]
    table_filter.drop([12,13,14], axis=1, inplace=True)
    taxonomy_bacteria=table_filter[table_filter[2]=="Bacteria"]
    #taxonomy_bacteria[3]=taxonomy_bacteria[3].apply(lambda x: "unknown" if x == "" else x)
    taxonomy_bacteria.drop([0,1], axis=1, inplace=True)
    return taxonomy_bacteria

In [382]:
soil_pks_megan=prepare_megan(megan_soil_pks)
soil_nrps_megan=prepare_megan(megan_soil_nrps)
roots_pks_megan=prepare_megan(megan_roots_pks)
roots_nrps_megan=prepare_megan(megan_roots_nrps)

In [3]:
megan_soil=pd.read_table("C:/Users/barakd/Desktop/metagenomes/soil_total_random_taxonomy.txt")
megan_roots=pd.read_table("C:/Users/barakd/Desktop/metagenomes/roots_total_random_taxonomy.txt")

In [4]:
soil_tax_total=prepare_megan(megan_soil)

C:\Users\barakd\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [13]:
soil_tax_total.set_index("query", inplace=True)
soil_tax_total=soil_tax_total.iloc[:, : 9]

In [17]:
roots_tax_total=prepare_megan(megan_roots)

C:\Users\barakd\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [18]:
roots_tax_total.set_index("query", inplace=True)
roots_tax_total=roots_tax_total.iloc[:, : 9]

In [20]:
roots_tax_total=roots_tax_total.iloc[:, : 7]
soil_tax_total=soil_tax_total.iloc[:, : 7]

In [23]:
roots_tax_total[3].value_counts()

Proteobacteria                      1745733
                                     923094
Terrabacteria group                  712066
Acidobacteria                        675752
FCB group                            460322
PVC group                            225133
unclassified Bacteria                 71540
Nitrospirae                           25268
Nitrospinae/Tectomicrobia group        9873
environmental samples <Bacteria>       9099
Spirochaetes                           3584
Elusimicrobia                          1131
Calditrichaeota                         332
Balneolaeota                            142
Rhodothermaeota                         117
Thermotogae <phylum>                     54
Thermodesulfobacteria <phylum>           43
Aquificae <phylum>                       41
Synergistetes                            30
Caldiserica                              20
Fusobacteria                             19
Deferribacteres <phylum>                 11
Chrysiogenetes <phylum>         

In [25]:
columns=["kingdom", "phyla", "class", "order","family","genus","species"]
soil_tax_total.columns=columns
roots_tax_total.columns=columns

In [31]:
soil_tax_total["class"].value_counts().to_clipboard()

In [21]:
soil_tax_total.to_csv("soil_tax_total.csv")
roots_tax_total.to_csv("roots_tax_total.csv")

# Merge the random PKS/NRPS hits with the taxonomy annotations:

In [383]:
print(roots_nrps_fltd.shape)
print(roots_nrps_megan.shape)


(7447, 12)
(49279, 18)


In [386]:
roots_nrps=roots_nrps_fltd.merge(roots_nrps_megan, on="query", how="left")
roots_pks=roots_pks_fltd.merge(roots_pks_megan, on="query", how="left")
soil_nrps=soil_nrps_fltd.merge(soil_nrps_megan, on="query", how="left")
soil_pks=soil_pks_fltd.merge(soil_pks_megan, on="query", how="left")

In [389]:
roots_nrps.to_csv("roots_nrps_tax.csv")
roots_pks.to_csv("roots_pks_tax.csv")
soil_nrps.to_csv("soil_nrps_tax.csv")
soil_pks.to_csv("soil_pks_tax.csv")

# Diversity Analysis

In [26]:
#Turn the hit table into a binary biom table based on accessions:

In [112]:
#groupbysample=roots_fltd.groupby("sample").nunique()
#groupbysample["query"]
groupbysample=soil_flted.groupby("sample").nunique()
groupbysample["query"]

sample
Amatitan_AD          737
MGS_AD               450
San_Felipe_AD        934
arctic_AD            389
chronofire_AD       1183
desert_AD            791
florida_lox_AD       434
iowa_prairie_AD      128
purdue_AD            342
subdeep_AD           448
tropical_peat_AD     364
Name: query, dtype: int64

Merge together the roots and soil results to create one biome table:
 

In [406]:
joined_pks=pd.concat([soil_pks_fltd, roots_pks_fltd])
joined_nrps=pd.concat([soil_nrps_fltd, roots_nrps_fltd])

In [407]:
groupbysample=joined_nrps.groupby("sample").nunique()
groupbysample["query"]

sample
Agave_orfs_random.fasta_AD           494
Amatitan_AD                          737
MGS_AD                               450
SM_exotic_orfs_random.fasta_AD       310
SM_native_orfs_random.fasta_AD       285
San_Felipe_AD                        934
arabidopsis_orfs_random.fasta_AD     948
arctic_AD                            389
carex_orfs_random.fasta_AD           278
chronofire_AD                       1183
cucumber_orfs_random.fasta_AD        423
desert_AD                            791
florida_lox_AD                       434
iowa_corn_orfs_random.fasta_AD       101
iowa_prairie_AD                      128
lettuce_orfs_random.fasta_AD         803
miscanthus_orfs_random.fasta_AD      114
paramo_orfs_random.fasta_AD          213
populus_orfs_random.fasta_AD         591
potato_orfs_random.fasta_AD          240
purdue_AD                            342
subdeep_AD                           448
switchgrass_orfs_random.fasta_AD     130
tomato_orfs_random.fasta_AD         1112
tropical_

In [408]:
def create_biom(df):
    df_grouped=df.groupby(["sample","accession"]).size().reset_index()
    df_grouped.columns=["sample", "accession", "hits"]
    biom=df_grouped.set_index(['sample', 'accession'])['hits'].unstack().reset_index().fillna(0)
    biom.set_index("sample", inplace=True)
    biom_binary=biom.apply(lambda x: [0 if y<1 else 1 for y in x])
    return biom_binary

In [142]:
#roots_grouped=roots_fltd.groupby(["sample","accession"]).size().reset_index()
#roots_grouped.columns=["sample", "accession", "hits"]
#biom_roots=roots_grouped.set_index(['sample', 'accession'])['hits'].unstack().reset_index().fillna(0)
#biom_roots.set_index("sample", inplace=True)

In [442]:
biom_nrps=create_biom(joined_nrps)
biom_pks=create_biom(joined_pks)

## Common and unique Genes- soil vs. roots (genes level) 

In [444]:
nrpsT=biom_nrps.T
nrpsT["sum"]=nrpsT.sum(axis=1)
nrpsT=nrpsT[nrpsT["sum"]>2]
nrpsT.drop("sum", axis=1, inplace=True)

In [455]:
nrpsT=nrpsT.T.reset_index()
nrpsT["env"]="roots"

In [460]:
nrpsT.to_csv("nrps_genes_over3.csv")

In [461]:
nrpsT=pd.read_csv("C:/Users/barakd/Desktop/metagenomes/nrps_genes_over3.csv")


In [462]:
nrps_genes_by_cat=nrpsT.groupby("env").sum().T

In [504]:
nrps_over_roots=nrps_genes_by_cat.loc[(nrps_genes_by_cat['roots'] >6) & (nrps_genes_by_cat['soil'] <4)]
nrps_over_soil=nrps_genes_by_cat.loc[(nrps_genes_by_cat['roots'] <4) & (nrps_genes_by_cat['soil'] >5)]
#nrps_over_roots.to_csv("nrps_over_roots_genes.csv")
#nrps_over_soil.to_csv("nrps_over_soil_genes.csv")

In [523]:
#nrps_over_roots.reset_index(inplace=True)
nrps_over_soil.reset_index(inplace=True)
nrps_over_soil.columns=["accession", "roots", "soil"]
nrps_over_soil_full=nrps_over_soil.merge(joined_nrps, on="accession", how="left")
nrps_over_soil_full.merge(soil_fltd, on="query", how="inner").to_csv("nrps_over_soil_with_mibig.csv")

In [524]:
#nrps_over_roots.columns=["accession", "roots", "soil"]
nrps_over_soil.columns=["accession", "roots", "soil"]

In [525]:
#nrps_over_roots_full=nrps_over_roots.merge(joined_nrps, on="accession", how="left")
nrps_over_soil_full=nrps_over_soil.merge(joined_nrps, on="accession", how="left")

In [526]:
#nrps_over_roots_full.merge(roots_fltd, on="query", how="inner").to_csv("nrps_over_roots_with_mibig.csv")
nrps_over_soil_full.merge(soil_fltd, on="query", how="inner").to_csv("nrps_over_soil_with_mibig.csv")

In [548]:
nrps_over_roots_full.to_csv("nrps_over_roots_genes_full.csv")
nrps_over_soil_full.to_csv("nrps_over_soil_genes_full.csv")

## do the same for PKS:

In [476]:
pksT=biom_pks.T
pksT["sum"]=pksT.sum(axis=1)
pksT=pksT[pksT["sum"]>2]
pksT.drop("sum", axis=1, inplace=True)

In [477]:
pksT=pksT.T.reset_index()
pksT["env"]="roots"
pksT.to_csv("pks_genes_over3.csv")

In [482]:
pksT=pd.read_csv("C:/Users/barakd/Desktop/metagenomes/pks_genes_over3.csv")
pks_genes_by_cat=pksT.groupby("env").sum().T

In [489]:
pks_over_roots=pks_genes_by_cat.loc[(pks_genes_by_cat['roots'] >5) & (pks_genes_by_cat['soil'] <4)]
pks_over_soil=pks_genes_by_cat.loc[(pks_genes_by_cat['roots'] <5) & (pks_genes_by_cat['soil'] >5)]
pks_over_roots.to_csv("pks_over_roots_genes.csv")
pks_over_soil.to_csv("pks_over_soil_genes.csv")

In [528]:
pks_over_roots.reset_index(inplace=True)
pks_over_soil.reset_index(inplace=True)

pks_over_roots.columns=["accession", "roots", "soil"]
pks_over_soil.columns=["accession", "roots", "soil"]

In [536]:
pks_over_roots_full=pks_over_roots.merge(joined_pks, on="accession", how="left")
pks_over_soil_full=pks_over_soil.merge(joined_pks, on="accession", how="left")


In [547]:
pks_over_soil_full.to_csv("pks_over_soil_genes_full.csv")
pks_over_roots_full.to_csv("pks_over_roots_genes_full.csv")

In [537]:
pks_over_soil_full.merge(soil_fltd, on="query", how="inner").to_csv("pks_over_soil_with_mibig.csv")
pks_over_roots_full.merge(roots_fltd, on="query", how="inner").to_csv("pks_over_roots_with_mibig.csv")

# MIBiG analyses (function-based)

Import the mibig diamond results of the randomly chosen ORFs for soil and roots:

In [543]:
mibig_roots=pd.read_table("C:/Users/barakd/Desktop/metagenomes/roots_mibig_results.txt")
mibig_soil=pd.read_table("C:/Users/barakd/Desktop/metagenomes/soil_mibig_results.txt")

In [544]:
columns=["query", "accession", "query_length","subject_length","alignment_length","query_start","query_end","identity", "eval", "bitscore","sample"]
mibig_roots.columns=columns
mibig_soil.columns=columns

In [545]:
#filter by alignment length:
mibig_roots_fltd=mibig_roots[mibig_roots["alignment_length"]>60]
mibig_soil_fltd=mibig_soil[mibig_soil["alignment_length"]>60]

#filter by evalue:
mibig_roots_fltd=mibig_roots_fltd[mibig_roots_fltd["eval"]<1e-30]
mibig_soil_fltd=mibig_soil_fltd[mibig_soil_fltd["eval"]<1e-30]

In [546]:
mibig_roots_fltd.shape
mibig_soil_fltd.shape

(783, 11)

In [293]:
#read the mibig products key:
mibig_key=pd.read_excel("C:/Users/barakd/Desktop/mibig_key.xlsx")

In [294]:
mibig_roots["accession"]=mibig_roots["accession"].str[:10]
mibig_soil["accession"]=mibig_soil["accession"].str[:10]

In [295]:
#merge the MIBIG key with the soil and roots contigs results (diamond to the mibig core):
roots_fltd=mibig_roots.merge(mibig_key,how="left", on="accession")
soil_fltd=mibig_soil.merge(mibig_key,how="left", on="accession")

In [222]:
roots_fltd.to_csv("mibig_roots_activity.csv")
soil_fltd.to_csv("mibig_soil_activity.csv")

In [296]:
roots_fltd.groupby("activity").nunique()/800*100

,query,accession,query_length,subject_length,alignment_length,query_start,query_end,identity,eval,bitscore,sample,product,bgc_class
activity,,,,,,,,,,,,,
NAP,36.500,8.375,20.000,16.500,18.500,3.000,19.125,19.000,34.125,27.625,1.375,7.750,0.750
antiameobe,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
antimicrobial,31.875,8.875,14.250,15.875,14.750,1.750,14.125,15.750,30.375,23.125,1.375,8.625,1.125
insecticidal,2.000,0.625,2.000,1.125,1.750,0.500,1.875,1.750,2.000,2.000,1.000,0.500,0.250
other,5.125,1.250,4.000,2.375,4.000,0.875,4.125,4.625,4.875,4.625,0.875,1.250,0.375
siderophore,7.875,1.250,6.500,1.750,6.375,0.750,6.875,7.375,7.875,7.625,1.000,1.250,0.125
surfactant,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
toxin,1.500,0.125,1.500,0.125,1.500,0.375,1.500,1.500,1.500,1.500,0.875,0.125,0.125
unknown,5.250,1.625,3.875,2.375,3.875,1.125,4.125,4.625,5.250,4.875,1.250,1.625,0.500


In [297]:
soil_fltd.groupby("activity").nunique()/783*100

,query,accession,query_length,subject_length,alignment_length,query_start,query_end,identity,eval,bitscore,sample,product,bgc_class
activity,,,,,,,,,,,,,
0,0.766284,0.510856,0.766284,0.510856,0.766284,0.255428,0.766284,0.766284,0.766284,0.766284,0.383142,0.510856,0.255428
NAP,23.243934,8.556833,10.600255,12.899106,10.472542,2.043423,11.238825,10.983397,22.477650,17.496807,1.277139,7.790549,0.638570
aniother,0.510856,0.127714,0.510856,0.255428,0.510856,0.127714,0.510856,0.510856,0.510856,0.510856,0.383142,0.127714,0.127714
antiameobe,0.127714,0.127714,0.127714,0.127714,0.127714,0.127714,0.127714,0.127714,0.127714,0.127714,0.127714,0.127714,0.127714
antimicrobial,25.542784,7.279693,11.366539,13.026820,11.749681,2.043423,11.749681,12.643678,24.137931,18.901660,1.277139,7.151980,0.893997
insecticidal,1.532567,0.510856,1.404853,1.149425,1.532567,0.766284,1.404853,1.404853,1.532567,1.532567,0.510856,0.383142,0.255428
other,23.243934,1.404853,17.752235,2.298851,17.496807,1.404853,17.879949,14.687101,22.733078,21.711367,1.277139,1.404853,0.383142
siderophore,2.171137,0.766284,2.043423,0.766284,1.915709,0.383142,1.915709,2.043423,2.171137,2.171137,0.766284,0.766284,0.255428
surfactant,0.127714,0.127714,0.127714,0.127714,0.127714,0.127714,0.127714,0.127714,0.127714,0.127714,0.127714,0.127714,0.127714


In [298]:
#Add the environment before the concatenation:
roots_fltd["env"]="roots"
soil_fltd["env"]="soil"

In [299]:
def create_biom_product(df):
    df_grouped=df.groupby(["sample","product"]).size().reset_index()
    df_grouped.columns=["sample", "product", "hits"]
    biom=df_grouped.set_index(['sample', 'product'])['hits'].unstack().reset_index().fillna(0)
    biom.set_index("sample", inplace=True)
    biom_binary=biom.apply(lambda x: [0 if y<1 else 1 for y in x])
    return biom_binary

In [300]:
joined=pd.concat([roots_fltd, soil_fltd])

In [301]:
mibig_biom=create_biom_product(joined)

In [241]:
mibig_biom.to_csv("public_mgs_mibig_biom.csv")

# Calculate common and unique products

In [302]:
mibigT=mibig_biom.T
mibigT["sum"]=mibigT.sum(axis=1)
#give only products that were found in 3 datasets and more:
mibigT=mibigT[mibigT["sum"]>2]
genes_by_category=genes_binary.groupby("env").sum().T

In [303]:
mibigT["sum"]=mibigT.sum(axis=1)

In [304]:
#give only products that were found in 3 datasets and more:
mibigT=mibigT[mibigT["sum"]>2]

In [305]:
mibigT.to_csv("mibig_products_table_eval.csv")

In [306]:
#Read it again after I added the category column:
genes_binary=pd.read_csv("C:/Users/barakd/Desktop/metagenomes/mibig_products_table_eval.csv")

In [307]:
genes_by_category=genes_binary.groupby("env").sum().T

In [308]:
genes_by_category

env,roots,soil
"1,3,6,8-tetrahydroxynaphthalene",2,1
A-74528,2,2
BE-43547 A1-C2,2,4
C-1027,1,2
E-837,3,3
...,...,...
tubulysin,1,2
versipelostatin,4,3
vicenistatin,0,3
vicibactin,4,0


In [315]:
over_roots=genes_by_category.loc[(genes_by_category['roots'] >4) & (genes_by_category['soil'] <5)]
over_soil=genes_by_category.loc[(genes_by_category['roots'] <5) & (genes_by_category['soil'] >4)]
over_roots.to_csv("over_roots_products_eval.csv")
over_soil.to_csv("over_soil_products_eval.csv")